In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np 
import pandas as pd

from pathlib import Path

In [3]:
train_dir = Path('E:\data\RSNA2024')

In [10]:
class CFG:
    random_seed = 42
    
    ROOT_FOLDER = train_dir
    IMAGES_DIR = ROOT_FOLDER / 'train_images'
    TRAIN_CSV = ROOT_FOLDER / 'train.csv'
    FILES_CSV = ROOT_FOLDER / 'train_files.csv'
    TRAIN_DESC_CSV = ROOT_FOLDER / 'train_series_descriptions.csv'
    COORDS_CSV = ROOT_FOLDER / 'train_label_coordinates.csv'

### Train data

In [5]:
train_df = pd.read_csv(CFG.TRAIN_CSV)
train_desc_df = pd.read_csv(CFG.TRAIN_DESC_CSV)

train_df.shape, train_desc_df.shape

((1975, 26), (6294, 3))

In [6]:
train_df.head()

,study_id,spinal_canal_stenosis_l1_l2,spinal_canal_stenosis_l2_l3,spinal_canal_stenosis_l3_l4,spinal_canal_stenosis_l4_l5,spinal_canal_stenosis_l5_s1,left_neural_foraminal_narrowing_l1_l2,left_neural_foraminal_narrowing_l2_l3,left_neural_foraminal_narrowing_l3_l4,left_neural_foraminal_narrowing_l4_l5,...,left_subarticular_stenosis_l1_l2,left_subarticular_stenosis_l2_l3,left_subarticular_stenosis_l3_l4,left_subarticular_stenosis_l4_l5,left_subarticular_stenosis_l5_s1,right_subarticular_stenosis_l1_l2,right_subarticular_stenosis_l2_l3,right_subarticular_stenosis_l3_l4,right_subarticular_stenosis_l4_l5,right_subarticular_stenosis_l5_s1
0,4003253,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
1,4646740,Normal/Mild,Normal/Mild,Moderate,Severe,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Normal/Mild,Normal/Mild,Moderate,Moderate,Moderate,Normal/Mild
2,7143189,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
3,8785691,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
4,10728036,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild


In [9]:
train_df.iloc[1]

study_id                                      4646740
spinal_canal_stenosis_l1_l2               Normal/Mild
spinal_canal_stenosis_l2_l3               Normal/Mild
spinal_canal_stenosis_l3_l4                  Moderate
spinal_canal_stenosis_l4_l5                    Severe
spinal_canal_stenosis_l5_s1               Normal/Mild
left_neural_foraminal_narrowing_l1_l2     Normal/Mild
left_neural_foraminal_narrowing_l2_l3     Normal/Mild
left_neural_foraminal_narrowing_l3_l4     Normal/Mild
left_neural_foraminal_narrowing_l4_l5        Moderate
left_neural_foraminal_narrowing_l5_s1        Moderate
right_neural_foraminal_narrowing_l1_l2    Normal/Mild
right_neural_foraminal_narrowing_l2_l3    Normal/Mild
right_neural_foraminal_narrowing_l3_l4       Moderate
right_neural_foraminal_narrowing_l4_l5       Moderate
right_neural_foraminal_narrowing_l5_s1    Normal/Mild
left_subarticular_stenosis_l1_l2          Normal/Mild
left_subarticular_stenosis_l2_l3          Normal/Mild
left_subarticular_stenosis_l

In [7]:
train_df.study_id.nunique()

1975

### Coordinates

In [28]:
coords_df = pd.read_csv(CFG.COORDS_CSV)
coords_df.shape

(48692, 7)

In [29]:
coords_df.study_id.nunique(), coords_df.condition.nunique(), coords_df.level.nunique()

(1974, 5, 5)

In [30]:
coords_df.condition.unique(), coords_df.level.unique()

(array(['Spinal Canal Stenosis', 'Right Neural Foraminal Narrowing',
        'Left Neural Foraminal Narrowing', 'Left Subarticular Stenosis',
        'Right Subarticular Stenosis'], dtype=object),
 array(['L1/L2', 'L2/L3', 'L3/L4', 'L4/L5', 'L5/S1'], dtype=object))

In [31]:
coords_df.series_id.nunique()

6291

In [32]:
coords_df['id'] = coords_df.apply(lambda row: str(row['study_id']) + str(row['series_id']), axis=1)
train_desc_df['id'] = train_desc_df.apply(lambda row: str(row['study_id']) + str(row['series_id']), axis=1)

In [33]:
coords_df.head(10)

,study_id,series_id,instance_number,condition,level,x,y,id
0,4003253,702807833,8,Spinal Canal Stenosis,L1/L2,322.831858,227.964602,4003253702807833
1,4003253,702807833,8,Spinal Canal Stenosis,L2/L3,320.571429,295.714286,4003253702807833
2,4003253,702807833,8,Spinal Canal Stenosis,L3/L4,323.030303,371.818182,4003253702807833
3,4003253,702807833,8,Spinal Canal Stenosis,L4/L5,335.292035,427.327434,4003253702807833
4,4003253,702807833,8,Spinal Canal Stenosis,L5/S1,353.415929,483.964602,4003253702807833
5,4003253,1054713880,4,Right Neural Foraminal Narrowing,L4/L5,187.961759,251.839388,40032531054713880
6,4003253,1054713880,4,Right Neural Foraminal Narrowing,L5/S1,198.240918,285.613767,40032531054713880
7,4003253,1054713880,5,Right Neural Foraminal Narrowing,L3/L4,187.227533,210.722753,40032531054713880
8,4003253,1054713880,6,Right Neural Foraminal Narrowing,L1/L2,194.569790,127.755258,40032531054713880
9,4003253,1054713880,6,Right Neural Foraminal Narrowing,L2/L3,191.632887,165.934990,40032531054713880


In [34]:
# coords_df[coords_df.study_id == 4003253]

In [35]:
train_desc_df[train_desc_df['id'] == '4003253702807833'].series_description.values[0]

'Sagittal T2/STIR'

In [36]:
coords_df['plane'] = coords_df.apply(lambda row: train_desc_df[train_desc_df['id'] == row['id']].series_description.values[0], axis=1)

In [37]:
coords_df.sample(5)

,study_id,series_id,instance_number,condition,level,x,y,id,plane
8515,767443105,3736569943,5,Left Neural Foraminal Narrowing,L1/L2,252.744155,94.563591,7674431053736569943,Sagittal T1
21877,1972541574,1380982312,9,Spinal Canal Stenosis,L1/L2,242.554945,129.406593,19725415741380982312,Sagittal T2/STIR
1737,155064828,1080671136,26,Left Subarticular Stenosis,L1/L2,168.269231,162.762951,1550648281080671136,Axial T2
22419,2019892566,2390719503,14,Right Subarticular Stenosis,L3/L4,170.238745,194.095498,20198925662390719503,Axial T2
43609,3856886497,2401411390,8,Spinal Canal Stenosis,L2/L3,218.619469,155.610619,38568864972401411390,Sagittal T2/STIR


In [38]:
# check canal stenosis is noy only in axial plane
coords_df[(coords_df.condition == 'Spinal Canal Stenosis') & (coords_df.plane != 'Axial T2')].sample()

,study_id,series_id,instance_number,condition,level,x,y,id,plane
41132,3617698707,2585122077,8,Spinal Canal Stenosis,L4/L5,275.483186,304.254867,36176987072585122077,Sagittal T2/STIR


In [39]:
# get the positive slices
coords_df.groupby(['study_id','series_id']).instance_number.unique()

study_id    series_id 
4003253     702807833                              [8]
            1054713880               [4, 5, 6, 11, 12]
            2448190387          [3, 4, 11, 19, 28, 35]
4646740     3201256954    [15, 16, 22, 28, 29, 34, 40]
            3486248476           [5, 6, 7, 15, 16, 17]
                                      ...             
4287160193  1507070277                             [8]
            1820446240          [4, 9, 10, 16, 22, 28]
4290709089  3274612423                             [9]
            3390218084    [2, 3, 5, 6, 10, 15, 20, 21]
            4237840455                  [4, 5, 11, 12]
Name: instance_number, Length: 6291, dtype: object

In [56]:
pos_slices = coords_df.groupby(['study_id','series_id']).instance_number.unique().apply(list).reset_index(name='slice').explode('slice')
pos_slices.shape

(24546, 3)

In [57]:
pos_slices

,study_id,series_id,slice
0,4003253,702807833,8
1,4003253,1054713880,4
1,4003253,1054713880,5
1,4003253,1054713880,6
1,4003253,1054713880,11
...,...,...,...
6289,4290709089,3390218084,21
6290,4290709089,4237840455,4
6290,4290709089,4237840455,5
6290,4290709089,4237840455,11


In [62]:
pos_slices.groupby('study_id').slice.nunique().mean()/3

3.6653157716987503

### train_df

In [42]:
# look at categories
for f in ['condition','level']:
    print(coords_df[f].value_counts())
    print('-'*50);print();

condition
Left Neural Foraminal Narrowing     9860
Right Neural Foraminal Narrowing    9859
Spinal Canal Stenosis               9753
Right Subarticular Stenosis         9612
Left Subarticular Stenosis          9608
Name: count, dtype: int64
--------------------------------------------------

level
L3/L4    9858
L4/L5    9858
L5/S1    9845
L2/L3    9661
L1/L2    9470
Name: count, dtype: int64
--------------------------------------------------



In [43]:
pd.crosstab(coords_df.condition, coords_df.level)

level,L1/L2,L2/L3,L3/L4,L4/L5,L5/S1
condition,,,,,
Left Neural Foraminal Narrowing,1972,1972,1972,1972,1972
Left Subarticular Stenosis,1810,1892,1971,1971,1964
Right Neural Foraminal Narrowing,1972,1972,1971,1972,1972
Right Subarticular Stenosis,1812,1891,1971,1971,1967
Spinal Canal Stenosis,1904,1934,1973,1972,1970


### Files

In [12]:
files_df = pd.read_csv(CFG.FILES_CSV)
files_df.shape

(147218, 5)

In [13]:
files_df.head(3)

,patient,series,image,rows,columns
0,100206310,1012284084,1,320,320
1,100206310,1012284084,10,320,320
2,100206310,1012284084,11,320,320


In [18]:
files_df.image.max(), files_df.image.mean()

(5049, 16.63592087924031)

In [47]:
# file names do not correspond to file count
files_df[files_df.image == 5049]

,patient,series,image,rows,columns
59719,2581283971,2683794967,5049,320,320


In [46]:
# max/mean images per patient
files_df.groupby(['patient','series']).image.count().max(), files_df.groupby(['patient','series']).image.count().mean()

(192, 23.390212901175722)

In [55]:
# mean positive imgs per series
coords_df.groupby(['study_id','series_id']).instance_number.nunique().mean()

3.9017644253695756